In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 10.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall:

In [19]:
import numpy as np, pandas as pd, torch, torch.nn as nn, torch.nn.functional as F
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from captum.attr import GradientShap
from sklearn.metrics import accuracy_score, matthews_corrcoef


In [2]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


# -------------------------------------------------------
# Dataset paths
# -------------------------------------------------------
DATASETS = {
    "chimpanzee": "/kaggle/input/dna-sequence-dataset/chimpanzee.txt",
    "human": "/kaggle/input/dna-sequence-dataset/human.txt",
    "dog": "/kaggle/input/dna-sequence-dataset/dog.txt"
}

MODEL_PATHS = {
    "chimpanzee": "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/LR_best_chimp.pkl",
    "human":       "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/LR_best_human.pkl",
    "dog":         "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/LR_best_dog.pkl"
}

TARGET_CLASS = 0     # GPCRs
K = 6                # k-mer size



# -------------------------------------------------------
# Load dataset
# -------------------------------------------------------
def load_dataset(name):
    df = pd.read_csv(DATASETS[name], sep="\t")
    df.columns = ["sequence", "class"]
    df["label"] = df["class"].astype("category").cat.codes
    return df


# -------------------------------------------------------
# K-mer extraction
# -------------------------------------------------------
def kmers(seq, k=6):
    seq = seq.lower()
    return [seq[i:i+k] for i in range(len(seq)-k+1)]



# -------------------------------------------------------
# Extract top-10 LR motifs
# -------------------------------------------------------
def extract_top10_lr_motifs(dataset_name, model_path, k=6, target_class=0):

    # Load dataset
    df = load_dataset(dataset_name)

    # Build text from kmers
    df["text"] = df["sequence"].apply(lambda s: " ".join(kmers(s, k)))
    texts = df["text"].values
    labels = df["label"].values

    # Train-test split for vectorizer consistency
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Fit vectorizer on whole dataset
    cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    cv.fit(texts)
    feature_names = np.array(cv.get_feature_names_out())

    # Load LR model
    LR = joblib.load(model_path)

    # Extract LR coefficients for class 0
    coef = LR.coef_[target_class]

    # Pick top-10 most positive (most predictive)
    idx = np.argsort(coef)[::-1][:10]
    motifs = feature_names[idx].tolist()
    weights = coef[idx].tolist()

    return motifs, weights



# -------------------------------------------------------
# Run Stage-1 analysis for Human, Chimpanzee, Dog
# -------------------------------------------------------
results = {}

for ds in ["human", "chimpanzee", "dog"]:
    motifs, weights = extract_top10_lr_motifs(
        dataset_name=ds,
        model_path=MODEL_PATHS[ds],
        k=K,
        target_class=TARGET_CLASS
    )
    
    results[ds] = {
        "motifs": motifs,
        "weights": weights
    }



# -------------------------------------------------------
# Print individual species top-10 motif lists
# -------------------------------------------------------
for ds in results:
    print(f"\n========================")
    print(f"Top-10 GPCR Motifs → {ds.upper()}")
    print("========================")
    for m, w in zip(results[ds]["motifs"], results[ds]["weights"]):
        print(f"{m}   |   weight={w:.4f}")



# -------------------------------------------------------
# Cross-species table (Human / Chimp / Dog)
# -------------------------------------------------------
top10_table = pd.DataFrame({
    "Human":       results["human"]["motifs"],
    "Chimpanzee":  results["chimpanzee"]["motifs"],
    "Dog":         results["dog"]["motifs"]
})

print("\n\nCROSS-SPECIES TOP-10 MOTIF COMPARISON TABLE:")
print(top10_table)



# -------------------------------------------------------
# CONSENSUS MOTIF ANALYSIS (Strict + Majority)
# -------------------------------------------------------

# Convert motif lists to sets for easy set operations:
human_set = set(results["human"]["motifs"])
chimp_set = set(results["chimpanzee"]["motifs"])
dog_set   = set(results["dog"]["motifs"])

# Strict: Appears in ALL 3 species
strict_consensus = human_set & chimp_set & dog_set

# Majority: Appears in at least 2 species
majority_consensus = (
    (human_set & chimp_set) |
    (human_set & dog_set)   |
    (chimp_set & dog_set)
)

print("\n\n========================")
print("STRICT CONSENSUS MOTIFS (appear in ALL 3 species):")
print("========================")
print(sorted(list(strict_consensus)) if strict_consensus else "None")


print("\n========================")
print("MAJORITY CONSENSUS MOTIFS (appear in ≥ 2 species):")
print("========================")
print(sorted(list(majority_consensus)) if majority_consensus else "None")




Top-10 GPCR Motifs → HUMAN
tcgctg   |   weight=0.2166
gtcctg   |   weight=0.2010
aacaac   |   weight=0.1985
agccag   |   weight=0.1940
gctgtg   |   weight=0.1936
cagcca   |   weight=0.1933
tgctct   |   weight=0.1840
cttctg   |   weight=0.1839
tgcggc   |   weight=0.1799
ttttca   |   weight=0.1752

Top-10 GPCR Motifs → CHIMPANZEE
gctgtg   |   weight=0.2060
gcttcc   |   weight=0.2033
tcatct   |   weight=0.2020
tcttca   |   weight=0.1904
catcat   |   weight=0.1787
tcgtgg   |   weight=0.1752
ccctgg   |   weight=0.1748
tgacca   |   weight=0.1717
cctggg   |   weight=0.1709
tgcctg   |   weight=0.1679

Top-10 GPCR Motifs → DOG
cctgct   |   weight=0.2274
ctctac   |   weight=0.2260
acctgg   |   weight=0.2252
ttcatt   |   weight=0.2086
cctggc   |   weight=0.2061
ctgctc   |   weight=0.2004
tcttct   |   weight=0.1834
cctggg   |   weight=0.1779
tcatct   |   weight=0.1687
ttgttt   |   weight=0.1659


CROSS-SPECIES TOP-10 MOTIF COMPARISON TABLE:
    Human Chimpanzee     Dog
0  tcgctg     gctgtg  cctgc

In [4]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


# -------------------------------------------------------
# Dataset + model paths
# -------------------------------------------------------
DATASET = "/kaggle/input/dna-sequence-dataset/human.txt"

MODEL_PATHS = {
    "LR":  "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/LR_best_human.pkl",
    "RF":  "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/RF_best_human.pkl",
    "MNB": "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/MNB_best_human.pkl"
}

TARGET_CLASS = 4   # Synthase
K = 6              # 6-mer features


# -------------------------------------------------------
# Load Human dataset
# -------------------------------------------------------
def load_dataset(path):
    df = pd.read_csv(path, sep="\t")
    df.columns = ["sequence", "class"]
    df["label"] = df["class"].astype("category").cat.codes
    return df


# -------------------------------------------------------
# k-mer extraction
# -------------------------------------------------------
def kmers(seq, k=6):
    seq = seq.lower()
    return [seq[i:i+k] for i in range(len(seq)-k+1)]


# -------------------------------------------------------
# Extract top-10 motifs for any ML model
# -------------------------------------------------------
def get_top10(model, feature_names, target_class):

    # Logistic Regression (coef_)
    if hasattr(model, "coef_"):
        fi = model.coef_[target_class]

    # Multinomial Naive Bayes (feature_log_prob_)
    elif hasattr(model, "feature_log_prob_"):
        fi = model.feature_log_prob_[target_class]

    # Random Forest (feature_importances_)
    else:
        fi = model.feature_importances_

    idx = np.argsort(fi)[::-1][:10]
    motifs = feature_names[idx].tolist()
    weights = fi[idx].tolist()
    return motifs, weights


# -------------------------------------------------------
# Build CountVectorizer and feature list
# -------------------------------------------------------
df = load_dataset(DATASET)
df["text"] = df["sequence"].apply(lambda s: " ".join(kmers(s, K)))

texts = df["text"].values
labels = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
cv.fit(texts)
feature_names = np.array(cv.get_feature_names_out())


# -------------------------------------------------------
# Extract motifs for LR, RF, MNB
# -------------------------------------------------------
results = {}

for model_name in MODEL_PATHS:
    model = joblib.load(MODEL_PATHS[model_name])
    motifs, weights = get_top10(model, feature_names, TARGET_CLASS)

    results[model_name] = {
        "motifs": motifs,
        "weights": weights
    }


# -------------------------------------------------------
# Print the results
# -------------------------------------------------------
for m in ["LR", "RF", "MNB"]:
    print(f"\n========================")
    print(f"Top-10 Synthase (Class 4) Motifs → {m}")
    print("========================")
    for motif, w in zip(results[m]["motifs"], results[m]["weights"]):
        print(f"{motif}   |   weight={w:.4f}")


# -------------------------------------------------------
# Build model comparison table
# -------------------------------------------------------
compare_table = pd.DataFrame({
    "LR":  results["LR"]["motifs"],
    "RF":  results["RF"]["motifs"],
    "MNB": results["MNB"]["motifs"]
})

print("\n\nCROSS-MODEL TOP-10 MOTIF COMPARISON TABLE (Human, Synthase):")
print(compare_table)


# -------------------------------------------------------
# Consensus motif analysis (strict + majority)
# -------------------------------------------------------
LR_set  = set(results["LR"]["motifs"])
RF_set  = set(results["RF"]["motifs"])
MNB_set = set(results["MNB"]["motifs"])

strict_consensus = LR_set & RF_set & MNB_set
majority_consensus = (
    (LR_set & RF_set) |
    (LR_set & MNB_set) |
    (RF_set & MNB_set)
)

print("\n========================")
print("STRICT CONSENSUS (in ALL 3 ML models):")
print("========================")
print(sorted(list(strict_consensus)) if strict_consensus else "None")

print("\n========================")
print("MAJORITY CONSENSUS (in ≥ 2 ML models):")
print("========================")
print(sorted(list(majority_consensus)) if majority_consensus else "None")



Top-10 Synthase (Class 4) Motifs → LR
ctggag   |   weight=0.2636
tgtccg   |   weight=0.2563
acattg   |   weight=0.2516
gtgggg   |   weight=0.2501
ctgtgc   |   weight=0.2411
gacaga   |   weight=0.2388
ggttct   |   weight=0.2360
tggggc   |   weight=0.2260
ttggaa   |   weight=0.2254
ggtact   |   weight=0.2212

Top-10 Synthase (Class 4) Motifs → RF
cagcag   |   weight=0.0029
agcagc   |   weight=0.0020
tgctgg   |   weight=0.0019
ggcatg   |   weight=0.0018
ctggtg   |   weight=0.0018
ggatgg   |   weight=0.0017
ccagca   |   weight=0.0017
tggtgt   |   weight=0.0015
gtgctg   |   weight=0.0015
ctctac   |   weight=0.0015

Top-10 Synthase (Class 4) Motifs → MNB
ctggag   |   weight=-6.4281
ctgctg   |   weight=-6.5712
ctgcag   |   weight=-6.6703
cctgga   |   weight=-6.6816
tggtgg   |   weight=-6.6930
tcctgg   |   weight=-6.7029
tgctgg   |   weight=-6.7112
gctgga   |   weight=-6.7348
cagctg   |   weight=-6.7643
gctgct   |   weight=-6.7875


CROSS-MODEL TOP-10 MOTIF COMPARISON TABLE (Human, Synthase):

In [5]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
import re

# ============================================================
# Stage 3 Configuration
# ============================================================
K = 6   # k-mer size
RANDOM_STATE = 42
FUNCTIONAL_GROUPS = list(range(7))  # labels 0–6

MODEL_PATHS = {
    "LR":  "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/LR_best_combined.pkl",
    "RF":  "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/RF_best_combined.pkl",
    "MNB": "/kaggle/input/dna-saved-models/pytorch/default/1/ML Models/ML Models/MNB_best_combined.pkl"
}

# ============================================================
# Step 1 — Load & Combine Dataset
# ============================================================
chimp = pd.read_csv("/kaggle/input/dna-sequence-dataset/chimpanzee.txt", sep="\t").dropna()
human = pd.read_csv("/kaggle/input/dna-sequence-dataset/human.txt", sep="\t").dropna()
dog   = pd.read_csv("/kaggle/input/dna-sequence-dataset/dog.txt", sep="\t").dropna()

combined = pd.concat([chimp, human, dog], ignore_index=True)
combined.columns = ["sequence", "class"]
combined["label"] = combined["class"].astype("category").cat.codes

print("Combined dataset size =", len(combined))


# ============================================================
# Step 2 — Utility: k-mer conversion
# ============================================================
def kmers(seq, k=6):
    seq = seq.lower()
    return [seq[i:i+k] for i in range(len(seq)-k+1)]


combined["text"] = combined["sequence"].apply(lambda s: " ".join(kmers(s, K)))


# ============================================================
# Step 3 — Vectorizer (fit on full combined dataset)
# ============================================================
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
cv.fit(combined["text"].values)
feature_names = np.array(cv.get_feature_names_out())

print("Vectorizer vocabulary size =", len(feature_names))


# ============================================================
# Step 4 — Train-test split (test ONLY for evaluation)
# ============================================================
X_train_text, X_test_text, y_train, y_test, seq_train, seq_test = train_test_split(
    combined["text"].values,
    combined["label"].values,
    combined["sequence"].values,
    test_size=0.2,
    stratify=combined["label"].values,
    random_state=RANDOM_STATE
)

X_test_vec = cv.transform(X_test_text)


# ============================================================
# Step 5 — Model loader
# ============================================================
models = {
    name: joblib.load(path)
    for name, path in MODEL_PATHS.items()
}

print("Loaded models:", models.keys())


# ============================================================
# Step 6 — Extract top-10 motifs for each model & functional group
# ============================================================
def get_top10_motifs(model, class_id):
    if hasattr(model, "coef_"):     # LR
        fi = model.coef_[class_id]
    elif hasattr(model, "feature_log_prob_"):  # MNB
        fi = model.feature_log_prob_[class_id]
    else:                           # RF
        fi = model.feature_importances_

    idx = np.argsort(fi)[::-1][:10]
    return feature_names[idx].tolist(), fi[idx].tolist()


# ============================================================
# Step 7 — Masking function (NNNNNN replacement)
# ============================================================
def mask_motifs(sequence, motifs, k=6):
    seq = sequence.lower()
    for m in motifs:
        seq = re.sub(m, "N" * k, seq)
    return seq


# ============================================================
# Step 8 — Evaluation helper
# ============================================================
def evaluate(model, X_vec, y_true):
    y_pred = model.predict(X_vec)
    return {
        "acc": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average="macro"),
        "mcc": matthews_corrcoef(y_true, y_pred)
    }


# ============================================================
# Step 9 — Stability metrics (Jaccard, intersection, % overlap)
# ============================================================
def stability_metrics(setA, setB):
    inter = len(setA & setB)
    union = len(setA | setB)
    jaccard = inter / union if union > 0 else 0
    percent = inter / 10
    return inter, jaccard, percent


# ============================================================
# MAIN STAGE 3 LOOP
# ============================================================
for fg in FUNCTIONAL_GROUPS:
    print("\n\n" + "="*70)
    print(f"FUNCTIONAL GROUP {fg} — ANALYSIS")
    print("="*70)

    # ---------------------------------------------------
    # A. Extract motifs for LR, RF, MNB
    # ---------------------------------------------------
    top10 = {}
    for model_name, model in models.items():
        motifs, weights = get_top10_motifs(model, fg)
        top10[model_name] = motifs

    print("\nTop-10 motifs per model:")
    print(pd.DataFrame(top10))


    # ---------------------------------------------------
    # B. Stability metrics
    # ---------------------------------------------------
    LR_set  = set(top10["LR"])
    RF_set  = set(top10["RF"])
    MNB_set = set(top10["MNB"])

    stability = {
        "LR-RF":  stability_metrics(LR_set, RF_set),
        "LR-MNB": stability_metrics(LR_set, MNB_set),
        "RF-MNB": stability_metrics(RF_set, MNB_set)
    }

    print("\nStability (intersection, jaccard, percent overlap):")
    for pair, (inter, jac, pct) in stability.items():
        print(f"{pair}: inter={inter},  jaccard={jac:.3f},  overlap%={pct*100:.1f}")


    # ---------------------------------------------------
    # C. Baseline evaluation
    # ---------------------------------------------------
    print("\nEvaluating baseline performance...")
    baseline = {
        name: evaluate(model, X_test_vec, y_test)
        for name, model in models.items()
    }
    print(pd.DataFrame(baseline).T)


    # ---------------------------------------------------
    # D. Masking test set for each model
    # ---------------------------------------------------
    fidelity_results = {}

    for model_name, model in models.items():

        print(f"\nMasking for model = {model_name}")

        # Mask sequences using that model's motifs for this FG
        masked_sequences = [mask_motifs(seq, top10[model_name], k=K) for seq in seq_test]

        # Convert to k-mer text
        masked_texts = [" ".join(kmers(seq, K)) for seq in masked_sequences]
        masked_vec = cv.transform(masked_texts)

        # Re-evaluate after masking
        masked_eval = evaluate(model, masked_vec, y_test)

        # Fidelity drop
        fidelity_results[model_name] = {
            "acc_drop": baseline[model_name]["acc"] - masked_eval["acc"],
            "f1_drop":  baseline[model_name]["f1"] - masked_eval["f1"],
            "mcc_drop": baseline[model_name]["mcc"] - masked_eval["mcc"]
        }

    print("\nFidelity drops after masking:")
    print(pd.DataFrame(fidelity_results).T)


Combined dataset size = 6882
Vectorizer vocabulary size = 4556
Loaded models: dict_keys(['LR', 'RF', 'MNB'])


FUNCTIONAL GROUP 0 — ANALYSIS

Top-10 motifs per model:
       LR      RF     MNB
0  ggccac  tgctgg  ctgctg
1  tcgctg  cagcag  tgctgg
2  atgctc  ctgctg  tcctgg
3  tgctct  ctggtg  tgctgc
4  gtcgct  cctgct  gctgct
5  tattgt  ccagca  cctgct
6  cagtcg  agcagc  cctgga
7  cgctgc  cctggt  cttcct
8  agagag  accagc  cctggg
9  gggctg  ggcatg  ctgcag

Stability (intersection, jaccard, percent overlap):
LR-RF: inter=0,  jaccard=0.000,  overlap%=0.0
LR-MNB: inter=0,  jaccard=0.000,  overlap%=0.0
RF-MNB: inter=3,  jaccard=0.176,  overlap%=30.0

Evaluating baseline performance...
          acc        f1       mcc
LR   0.938272  0.939952  0.924467
RF   0.904139  0.910224  0.885401
MNB  0.660857  0.656981  0.601481

Masking for model = LR

Masking for model = RF

Masking for model = MNB

Fidelity drops after masking:
     acc_drop   f1_drop  mcc_drop
LR   0.003631  0.003687  0.004460
RF   0.01

In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split
from captum.attr import IntegratedGradients



DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ============================================================
# PARAMS
# ============================================================
K = 6
MAX_LEN = 300
BATCH = 32
FG = 0   # functional group to analyze (GPCR)

DATASETS = {
    "chimpanzee": "/kaggle/input/dna-sequence-dataset/chimpanzee.txt",
    "human":      "/kaggle/input/dna-sequence-dataset/human.txt",
    "dog":        "/kaggle/input/dna-sequence-dataset/dog.txt"
}

# CNN configs from your logs
CNN_CONFIG = {
    "chimpanzee": {"embed_dim":128, "num_filters":256, "kernel_sizes":(5,7,9)},
    "human":      {"embed_dim":128, "num_filters":256, "kernel_sizes":(5,7,9)},
    "dog":        {"embed_dim":256, "num_filters":256, "kernel_sizes":(3,5,7)}
}

MODEL_PATH = {
    "chimpanzee": "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/chimpanzee_kmer_CNN.pt",
    "human":      "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN.pt",
    "dog":        "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/dog_kmer_CNN.pt"
}

# ============================================================
# Utility functions (from your training pipeline)
# ============================================================
def kmers(seq, k=6):
    return [seq[i:i+k].lower() for i in range(len(seq)-k+1)]

def build_vocab(seqs):
    all_k = []
    for s in seqs:
        all_k.extend(kmers(s, K))
    return {k:i+1 for i,(k,_) in enumerate(Counter(all_k).most_common())}

def encode_kmer(seq, vocab):
    return [vocab[k] for k in kmers(seq, K) if k in vocab]

def pad_kmer(seqs):
    out = np.zeros((len(seqs), MAX_LEN), dtype=np.int64)
    for i, s in enumerate(seqs):
        out[i, :len(s[:MAX_LEN])] = s[:MAX_LEN]
    return out

class KmerDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ============================================================
# CNN MODEL (exact match to training)
# ============================================================
class CNN(nn.Module):
    def __init__(self, vocab_size, num_classes, embed_dim, num_filters, kernel_sizes, dropout=0.4):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList(
            [nn.Conv1d(embed_dim, num_filters, k, padding=k//2) for k in kernel_sizes]
        )
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        x = self.embed(x)               # (B,T,E)
        x = x.transpose(1,2)            # (B,E,T)
        h = [F.max_pool1d(F.relu(c(x)), x.shape[2]).squeeze(2)
             for c in self.convs]
        return self.fc(self.drop(torch.cat(h, 1)))

# ============================================================
# WRAPPER for IG (IG operates on embedded vectors, not indices)
# ============================================================
class CNN_Embedded(nn.Module):
    def __init__(self, cnn_model):
        super().__init__()
        self.cnn = cnn_model

    def forward(self, embedded_x):
        # embedded_x shape = (B,T,E)
        x = embedded_x.transpose(1,2)  # (B,E,T)
        h = [F.max_pool1d(F.relu(conv(x)), x.shape[2]).squeeze(2)
             for conv in self.cnn.convs]
        out = self.cnn.fc(self.cnn.drop(torch.cat(h,1)))
        return out

# ============================================================
# Load dataset
# ============================================================
def load_dataset(name):
    df = pd.read_csv(DATASETS[name], sep="\t")
    df.columns = ["sequence", "class"]
    df["label"] = df["class"].astype("category").cat.codes
    return df

# ============================================================
# Integrated Gradients → top10 motifs per dataset
# ============================================================
def get_top10_IG_for_dataset(ds_name):

    print("\n" + "="*70)
    print(f"DATASET: {ds_name.upper()} — CNN IG FOR FG={FG}")
    print("="*70)

    # -----------------------------
    # Load dataset
    # -----------------------------
    df = load_dataset(ds_name)
    X_raw = df["sequence"].values
    y = df["label"].values
    NUM_CLASSES = len(np.unique(y))

    # -----------------------------
    # Build vocab per dataset
    # -----------------------------
    vocab = build_vocab(X_raw)
    vocab_inv = {v:k for k,v in vocab.items()}

    # -----------------------------
    # Encode + pad
    # -----------------------------
    X_enc = [encode_kmer(s, vocab) for s in X_raw]
    X_pad = pad_kmer(X_enc)

    X_train, X_test, y_train, y_test = train_test_split(
        X_pad, y, test_size=0.2, stratify=y, random_state=42
    )

    # -----------------------------
    # Load trained CNN with correct config
    # -----------------------------
    cfg = CNN_CONFIG[ds_name]

    cnn = CNN(
        vocab_size=len(vocab)+1,
        num_classes=NUM_CLASSES,
        embed_dim=cfg["embed_dim"],
        num_filters=cfg["num_filters"],
        kernel_sizes=cfg["kernel_sizes"]
    ).to(DEVICE)

    cnn.load_state_dict(torch.load(MODEL_PATH[ds_name], map_location=DEVICE))
    cnn.eval()

    wrap = CNN_Embedded(cnn).to(DEVICE)
    ig = IntegratedGradients(wrap)

    # -----------------------------
    # Run IG only on FG samples
    # -----------------------------
    idx_fg = np.where(y_test == FG)[0]
    print(f"Test samples for FG={FG}: {len(idx_fg)}")

    total_attr = Counter()

    for idx in idx_fg:
        x = torch.tensor(X_test[idx], dtype=torch.long).unsqueeze(0).to(DEVICE)

        # embedded input
        emb = cnn.embed(x)                      # (1,T,E)
        baseline = torch.zeros_like(emb)        # baseline = zero embedding

        # IG attribution
        attr_emb, _ = ig.attribute(
            emb,
            baselines=baseline,
            target=FG,
            n_steps=50,
            return_convergence_delta=True
        )

        # reduce embedding dimension → token-level attribution
        token_scores = attr_emb.sum(dim=2).squeeze(0).detach().cpu().numpy()

        # accumulate per token-id
        for pos, score in enumerate(token_scores):
            token_id = int(X_test[idx][pos])
            if token_id != 0:
                total_attr[token_id] += float(score)

    # -----------------------------
    # Convert token IDs → k-mers
    # -----------------------------
    kmer_attr = {}
    for token_id, score in total_attr.items():
        if token_id in vocab_inv and score > 0:
            kmer_attr[vocab_inv[token_id]] = score

    # -----------------------------
    # Top-10 motifs
    # -----------------------------
    top10 = sorted(kmer_attr.items(), key=lambda x: x[1], reverse=True)[:10]

    motifs = [m for m,_ in top10]
    weights = [w for _,w in top10]

    return motifs, weights

# ============================================================
# RUN IG FOR ALL THREE DATASETS
# ============================================================
results = {}

for ds in ["human", "chimpanzee", "dog"]:
    motifs, weights = get_top10_IG_for_dataset(ds)
    results[ds] = {"motifs": motifs, "weights": weights}

# ============================================================
# PRINT INDIVIDUAL DATASET RESULTS
# ============================================================
for ds in results:
    print("\n========================")
    print(f"Top-10 IG Motifs → {ds.upper()}")
    print("========================")
    for m,w in zip(results[ds]["motifs"], results[ds]["weights"]):
        print(f"{m}   |   score={w:.4f}")

# ============================================================
# CROSS-SPECIES TABLE
# ============================================================
top10_table = pd.DataFrame({
    "Human": results["human"]["motifs"],
    "Chimpanzee": results["chimpanzee"]["motifs"],
    "Dog": results["dog"]["motifs"]
})

print("\n\nCROSS-SPECIES TOP-10 IG MOTIF COMPARISON TABLE:")
print(top10_table)

# ============================================================
# CONSENSUS ANALYSIS
# ============================================================
h = set(results["human"]["motifs"])
c = set(results["chimpanzee"]["motifs"])
d = set(results["dog"]["motifs"])

strict_consensus = h & c & d
majority_consensus = (h & c) | (h & d) | (c & d)

print("\n\n========================")
print("STRICT CONSENSUS MOTIFS (appear in ALL 3 species):")
print("========================")
print(list(strict_consensus) if strict_consensus else "None")

print("\n========================")
print("MAJORITY CONSENSUS MOTIFS (appear in ≥2 species):")
print("========================")
print(list(majority_consensus) if majority_consensus else "None")



DATASET: HUMAN — CNN IG FOR FG=0
Test samples for FG=0: 106

DATASET: CHIMPANZEE — CNN IG FOR FG=0
Test samples for FG=0: 47

DATASET: DOG — CNN IG FOR FG=0
Test samples for FG=0: 26

Top-10 IG Motifs → HUMAN
tgttcc   |   score=21.4962
cctgct   |   score=19.4643
ctgctc   |   score=19.3442
ctcttc   |   score=19.3343
tgctct   |   score=18.6367
ttcctg   |   score=18.2247
gttcct   |   score=16.3187
cttctg   |   score=16.1633
ctgctg   |   score=14.4151
tcttct   |   score=12.9497

Top-10 IG Motifs → CHIMPANZEE
cctgct   |   score=10.4112
ctgctc   |   score=10.2236
tgctct   |   score=6.7357
ctcttc   |   score=5.6141
ccctgc   |   score=5.4651
ctcctg   |   score=4.5408
tgctcc   |   score=4.3208
ctgctg   |   score=4.1764
gctcct   |   score=3.7282
ctggcc   |   score=3.5170

Top-10 IG Motifs → DOG
gctgct   |   score=3.7453
ctgctc   |   score=3.4347
tcctgg   |   score=3.0909
tgctgc   |   score=3.0881
cctgct   |   score=2.3990
ctgctg   |   score=2.3687
cttcct   |   score=2.3247
ctgcct   |   score=2.

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split
from captum.attr import IntegratedGradients
import torch
torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ============================================================
# PARAMS
# ============================================================
K = 6
MAX_LEN = 300
BATCH = 32
FG = 6   # Transcription Factor

DATASET = "/kaggle/input/dna-sequence-dataset/human.txt"

# ============================================================
# DL CONFIGS FOR HUMAN DATASET (from your logs)
# ============================================================
CNN_CFG =       {"embed_dim":128, "num_filters":256, "kernel_sizes":(5,7,9)}
ATTN_CFG =      {"embed_dim":256, "conv_filters":256, "conv_kernel":15, "n_heads":4}
BI_LSTM_CFG =   {"embed_dim":128, "conv_filters":256, "lstm_hidden":256}

MODEL_PATHS = {
    "CNN":          "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN.pt",
    "CNN-Attn":     "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN-Attention.pt",
    "CNN-BiLSTM":   "/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN-BiLSTM.pt"
}

# ============================================================
# Utility functions (same as training)
# ============================================================
def kmers(seq, k=6):
    return [seq[i:i+k].lower() for i in range(len(seq)-k+1)]

def build_vocab(seqs):
    all_k = []
    for s in seqs:
        all_k.extend(kmers(s, K))
    return {k:i+1 for i,(k,_) in enumerate(Counter(all_k).most_common())}

def encode_kmer(seq, vocab):
    return [vocab[k] for k in kmers(seq, K) if k in vocab]

def pad_kmer(seqs):
    out = np.zeros((len(seqs), MAX_LEN), dtype=np.int64)
    for i,s in enumerate(seqs):
        out[i,:len(s[:MAX_LEN])] = s[:MAX_LEN]
    return out

class KmerDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ============================================================
# Three DL models for analysis
# ============================================================
class CNN(nn.Module):
    def __init__(self, vocab_size, num_classes, embed_dim, num_filters, kernel_sizes, dropout=0.4):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([nn.Conv1d(embed_dim, num_filters, k, padding=k//2) for k in kernel_sizes])
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        x = self.embed(x)
        x = x.transpose(1,2)
        h = [F.max_pool1d(F.relu(c(x)), x.shape[2]).squeeze(2) for c in self.convs]
        return self.fc(self.drop(torch.cat(h,1)))

class CNNAttention(nn.Module):
    def __init__(self,vocab_size,num_classes,embed_dim,conv_filters,conv_kernel,n_heads,dropout=0.4):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim,conv_filters,conv_kernel,padding=conv_kernel//2)
        self.proj = nn.Linear(conv_filters,embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim,n_heads,batch_first=True)
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(embed_dim,num_classes)

    def forward(self,x):
        x = self.embed(x)
        x = F.relu(self.conv(x.transpose(1,2))).transpose(1,2)
        x = self.proj(x)
        a,_ = self.attn(x,x,x)
        return self.fc(self.drop(a.mean(1)))

class CNNBiLSTM(nn.Module):
    def __init__(self,vocab_size,num_classes,embed_dim,conv_filters,lstm_hidden,dropout=0.3):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim,conv_filters,7,padding=3)
        self.lstm = nn.LSTM(conv_filters,lstm_hidden,batch_first=True,bidirectional=True)
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_hidden * 2,num_classes)

    def forward(self,x):
        x = self.embed(x)
        x = F.relu(self.conv(x.transpose(1,2))).transpose(1,2)
        h,_ = self.lstm(x)
        h,_ = torch.max(h,1)
        return self.fc(self.drop(h))

# ============================================================
# Wrapper for IG (operates on embedded tensors)
# ============================================================
class WrappedModel(nn.Module):
    def __init__(self, base_model, model_type):
        super().__init__()
        self.base = base_model
        self.model_type = model_type

    def forward(self, embedded_x):
        if self.model_type == "CNN":
            x = embedded_x.transpose(1,2)
            h = [F.max_pool1d(F.relu(conv(x)), x.shape[2]).squeeze(2) 
                 for conv in self.base.convs]
            return self.base.fc(self.base.drop(torch.cat(h,1)))

        elif self.model_type == "CNN-Attn":
            x = embedded_x.transpose(1,2)
            x = F.relu(self.base.conv(x)).transpose(1,2)
            x = self.base.proj(x)
            a,_ = self.base.attn(x,x,x)
            return self.base.fc(self.base.drop(a.mean(1)))

        else:  # CNN-BiLSTM
            x = embedded_x.transpose(1,2)
            x = F.relu(self.base.conv(x)).transpose(1,2)
            h,_ = self.base.lstm(x)
            h,_ = torch.max(h,1)
            return self.base.fc(self.base.drop(h))

# ============================================================
# IG extraction for Transcription Factor (FG=6)
# ============================================================
def get_top10_IG(model_name, cfg, model_path):

    print(f"\n\n============================")
    print(f"MODEL: {model_name}")
    print("============================")

    # -------------------
    # Load dataset
    # -------------------
    df = pd.read_csv(DATASET, sep="\t")
    df.columns = ["sequence", "class"]
    df["label"] = df["class"].astype("category").cat.codes
    X_raw = df["sequence"].values
    y = df["label"].values
    NUM_CLASSES = len(np.unique(y))

    # -------------------
    # Build vocab
    # -------------------
    vocab = build_vocab(X_raw)
    vocab_inv = {v:k for k,v in vocab.items()}

    X_enc = [encode_kmer(s,vocab) for s in X_raw]
    X_pad = pad_kmer(X_enc)

    X_train,X_test,y_train,y_test = train_test_split(
        X_pad,y,test_size=0.2,stratify=y,random_state=42
    )

    # -------------------
    # Load correct model
    # -------------------
    if model_name == "CNN":
        base_model = CNN(len(vocab)+1, NUM_CLASSES, **cfg).to(DEVICE)

    elif model_name == "CNN-Attn":
        base_model = CNNAttention(len(vocab)+1, NUM_CLASSES, **cfg).to(DEVICE)

    else:
        base_model = CNNBiLSTM(len(vocab)+1, NUM_CLASSES, **cfg).to(DEVICE)

    base_model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    base_model.eval()

    # -------------------
    # IG wrapper
    # -------------------
    wrap = WrappedModel(base_model, model_name).to(DEVICE)
    ig = IntegratedGradients(wrap)

    # -------------------
    # Use only TF class samples
    # -------------------
    idx_fg = np.where(y_test == FG)[0]
    print(f"TF samples in test set: {len(idx_fg)}")

    total_attr = Counter()

    # -------------------
    # IG loop
    # -------------------
    for idx in idx_fg:
        x = torch.tensor(X_test[idx], dtype=torch.long).unsqueeze(0).to(DEVICE)

        embedded = base_model.embed(x)
        baseline = torch.zeros_like(embedded)

        attr_emb,_ = ig.attribute(
            embedded,
            baselines=baseline,
            target=FG,
            n_steps=40,
            return_convergence_delta=True
        )

        scores = attr_emb.sum(dim=2).squeeze(0).detach().cpu().numpy()

        for pos,score in enumerate(scores):
            token_id = int(X_test[idx][pos])
            if token_id != 0:
                total_attr[token_id] += float(score)

    # -------------------
    # Map back to k-mers
    # -------------------
    kmer_attr = {vocab_inv[t]:score for t,score in total_attr.items()
                 if t in vocab_inv and score > 0}

    top10 = sorted(kmer_attr.items(), key=lambda x: x[1], reverse=True)[:10]

    motifs = [m for m,_ in top10]
    weights = [w for _,w in top10]

    return motifs, weights

# ============================================================
# RUN FOR 3 MODELS
# ============================================================
results = {}

results["CNN"]       = get_top10_IG("CNN",       CNN_CFG,     MODEL_PATHS["CNN"])
results["CNN-Attn"]  = get_top10_IG("CNN-Attn",  ATTN_CFG,    MODEL_PATHS["CNN-Attn"])
results["CNN-BiLSTM"]= get_top10_IG("CNN-BiLSTM",BI_LSTM_CFG, MODEL_PATHS["CNN-BiLSTM"])

# ============================================================
# PRINT RESULTS EXACTLY LIKE ML VERSION
# ============================================================
for m in ["CNN","CNN-Attn","CNN-BiLSTM"]:
    print(f"\n========================")
    print(f"Top-10 TF (Class 6) Motifs → {m}")
    print("========================")
    motifs,weights = results[m]
    for motif,w in zip(motifs,weights):
        print(f"{motif}   |   score={w:.4f}")

# ============================================================
# COMPARISON TABLE
# ============================================================
compare_table = pd.DataFrame({
    "CNN":       results["CNN"][0],
    "CNN-Attn":  results["CNN-Attn"][0],
    "CNN-BiLSTM":results["CNN-BiLSTM"][0]
})

print("\n\nCROSS-MODEL TOP-10 MOTIF COMPARISON TABLE (Human, TF):")
print(compare_table)

# ============================================================
# CONSENSUS ANALYSIS
# ============================================================
CNN_set       = set(results["CNN"][0])
AT_set        = set(results["CNN-Attn"][0])
BI_set        = set(results["CNN-BiLSTM"][0])

strict_consensus = CNN_set & AT_set & BI_set
majority_consensus = (CNN_set & AT_set) | (CNN_set & BI_set) | (AT_set & BI_set)

print("\n========================")
print("STRICT CONSENSUS (ALL 3 DL models):")
print("========================")
print(sorted(list(strict_consensus)) if strict_consensus else "None")

print("\n========================")
print("MAJORITY CONSENSUS (≥2 DL models):")
print("========================")
print(sorted(list(majority_consensus)) if majority_consensus else "None")




MODEL: CNN
TF samples in test set: 269


MODEL: CNN-Attn
TF samples in test set: 269


MODEL: CNN-BiLSTM
TF samples in test set: 269

Top-10 TF (Class 6) Motifs → CNN
cagcag   |   score=76.8405
agcagc   |   score=53.1437
gcagca   |   score=32.3086
aggagg   |   score=25.3584
ggagga   |   score=23.4207
agcccc   |   score=22.3878
ccagca   |   score=21.2277
ccccgg   |   score=20.1202
gcggcg   |   score=19.8683
agcaga   |   score=19.0683

Top-10 TF (Class 6) Motifs → CNN-Attn
cagcag   |   score=171.9802
agcagc   |   score=101.3025
ccagca   |   score=77.1736
gcagca   |   score=67.2537
aggagg   |   score=58.4936
cacaga   |   score=54.7885
cggagg   |   score=44.4612
agctca   |   score=40.4150
ctggaa   |   score=37.5330
tggagg   |   score=35.8858

Top-10 TF (Class 6) Motifs → CNN-BiLSTM
agcagc   |   score=41.4212
atggcc   |   score=29.7714
cagcag   |   score=23.7486
atggac   |   score=21.9047
gcagca   |   score=21.3839
tggagg   |   score=15.7186
atggcg   |   score=13.0852
agaaga   |   score=1

In [12]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
import re

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =====================================================
# LOAD COMBINED DATASET
# =====================================================
chimp = pd.read_csv("/kaggle/input/dna-sequence-dataset/chimpanzee.txt", sep="\t").dropna()
human = pd.read_csv("/kaggle/input/dna-sequence-dataset/human.txt", sep="\t").dropna()
dog   = pd.read_csv("/kaggle/input/dna-sequence-dataset/dog.txt", sep="\t").dropna()

combined = pd.concat([chimp, human, dog], ignore_index=True)
combined.columns = ["sequence", "class"]
combined["label"] = combined["class"].astype("category").cat.codes

FUNCTIONAL_GROUPS = sorted(combined["label"].unique())  # [0..6]

# =====================================================
# VOCAB RECONSTRUCTION (PERFECT MATCH)
# =====================================================
K = 6
MAX_LEN = 300

def kmers(seq, k=K):
    seq = seq.lower()
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

def rebuild_vocab(seqs, k=K):
    all_k = []
    for s in seqs:
        all_k.extend(kmers(s, k))
    # EXACT ordering used during training
    return {km: idx+1 for idx,(km,_) in enumerate(Counter(all_k).most_common())}

vocab = rebuild_vocab(combined["sequence"].values)
VOCAB_SIZE = len(vocab) + 1
print("Reconstructed vocab size:", VOCAB_SIZE)

def encode(seq):
    return [vocab[k] for k in kmers(seq) if k in vocab]

def pad(seqs):
    out = np.zeros((len(seqs), MAX_LEN), dtype=np.int64)
    for i, seq in enumerate(seqs):
        s = seq[:MAX_LEN]
        out[i, :len(s)] = s
    return out

# =====================================================
# DATA SPLIT
# =====================================================
X_raw = combined["sequence"].values
y = combined["label"].values

X_enc = pad([encode(s) for s in X_raw])

X_train, X_test, y_train, y_test, X_raw_train, X_raw_test = train_test_split(
    X_enc, y, X_raw, test_size=0.2, stratify=y, random_state=42
)

NUM_CLASSES = len(np.unique(y))

# =====================================================
# DATASET CLASS
# =====================================================
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

test_loader = DataLoader(SeqDataset(X_test, y_test), batch_size=64)

# =====================================================
# DL MODELS — EXACT TRAINED CONFIGS
# =====================================================
class CNN(nn.Module):
    def __init__(self, vocab_size, nc, embed_dim=128, num_filters=256, kernel_sizes=(5,7,9)):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([nn.Conv1d(embed_dim, num_filters, k, padding=k//2) 
                                    for k in kernel_sizes])
        self.fc = nn.Linear(num_filters * len(kernel_sizes), nc)

    def forward(self, x):
        x = self.embed(x).transpose(1,2)
        h = [F.max_pool1d(F.relu(conv(x)), x.shape[2]).squeeze(2) for conv in self.convs]
        return self.fc(torch.cat(h,1))

class CNNAttn(nn.Module):
    def __init__(self, vocab_size, nc, embed_dim=128, conv_filters=128, conv_kernel=7, n_heads=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, conv_filters, conv_kernel, padding=conv_kernel//2)
        self.proj = nn.Linear(conv_filters, embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, n_heads, batch_first=True)
        self.fc = nn.Linear(embed_dim, nc)

    def forward(self,x):
        x = self.embed(x)
        x = F.relu(self.conv(x.transpose(1,2))).transpose(1,2)
        x = self.proj(x)
        a,_ = self.attn(x,x,x)
        return self.fc(a.mean(1))

class CNNBiLSTM(nn.Module):
    def __init__(self, vocab_size, nc, embed_dim=256, conv_filters=256, lstm_hidden=512):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, conv_filters, 7, padding=3)
        self.lstm = nn.LSTM(conv_filters, lstm_hidden, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(lstm_hidden*2, nc)

    def forward(self, x):
        x = self.embed(x)
        x = F.relu(self.conv(x.transpose(1,2))).transpose(1,2)
        h,_ = self.lstm(x)
        h,_ = torch.max(h,1)
        return self.fc(h)

# =====================================================
# LOAD SAVED MODELS
# =====================================================
CNN_CFG = {"embed_dim":128, "num_filters":256, "kernel_sizes":(5,7,9)}
ATTN_CFG = {"embed_dim":128, "conv_filters":128, "conv_kernel":7, "n_heads":2}
BI_CFG   = {'embed_dim': 128, 'conv_filters': 256, 'lstm_hidden': 256}

cnn = CNN(VOCAB_SIZE, NUM_CLASSES, **CNN_CFG).to(DEVICE)
attn = CNNAttn(VOCAB_SIZE, NUM_CLASSES, **ATTN_CFG).to(DEVICE)
bilstm = CNNBiLSTM(VOCAB_SIZE, NUM_CLASSES, **BI_CFG).to(DEVICE)

cnn.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/combined_kmer_CNN.pt", map_location=DEVICE))
attn.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/combined_kmer_CNN-Attention.pt", map_location=DEVICE))
bilstm.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/combined_kmer_CNN-BiLSTM.pt", map_location=DEVICE))

models = {"CNN": cnn, "CNN-Attn": attn, "CNN-BiLSTM": bilstm}

# =====================================================
# HELPER: FORWARD EMBEDDING PASS FOR IG
# =====================================================
def forward_emb(model, emb):
    if isinstance(model, CNN):
        x = emb.transpose(1,2)
        h = [F.max_pool1d(F.relu(c(x)), x.shape[2]).squeeze(2) for c in model.convs]
        return model.fc(torch.cat(h,1))

    elif isinstance(model, CNNAttn):
        x = F.relu(model.conv(emb.transpose(1,2))).transpose(1,2)
        x = model.proj(x)
        a,_ = model.attn(x,x,x)
        return model.fc(a.mean(1))

    else:  # CNN-BiLSTM
        x = F.relu(model.conv(emb.transpose(1,2))).transpose(1,2)
        h,_ = model.lstm(x)
        h,_ = torch.max(h,1)
        return model.fc(h)

# =====================================================
# IG MOTIF EXTRACTION
# =====================================================
def get_top10_IG(model, FG):
    model.eval()

    embed = model.embed
    ig = IntegratedGradients(lambda emb: forward_emb(model, emb))

    FG_indices = np.where(y_test == FG)[0]
    if len(FG_indices) == 0: return [], []

    attr_sum = Counter()

    for idx in FG_indices[:200]:
        x = torch.tensor(X_test[idx], dtype=torch.long, device=DEVICE).unsqueeze(0)
        emb = embed(x)
        baseline = torch.zeros_like(emb)

        attr, _ = ig.attribute(
    emb,
    baselines=baseline,
    target=int(FG),
    return_convergence_delta=True
)

        attr = attr.squeeze(0).abs().sum(dim=1).detach().cpu()

        km = kmers(X_raw_test[idx])
        for score, kmer in zip(attr[:len(km)], km):
            attr_sum[kmer] += float(score)

    motifs, weights = zip(*attr_sum.most_common(10))
    return list(motifs), list(weights)

# =====================================================
# MASKING (OPTION A)
# =====================================================
def mask_sequences(raw_list, motifs):
    masked = []
    for seq in raw_list:
        sx = seq.lower()
        for m in motifs:
            sx = re.sub(m, "N"*K, sx)
        enc = encode(sx)
        enc = enc[:MAX_LEN]
        padded = enc + [0]*(MAX_LEN - len(enc))
        masked.append(padded)
    return np.array(masked, dtype=np.int64)

# =====================================================
# DL EVALUATION
# =====================================================
def eval_dl(model, X, y):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(0, len(X), 64):
            xb = torch.tensor(X[i:i+64], dtype=torch.long, device=DEVICE)
            logits = model(xb)
            preds.extend(logits.argmax(1).cpu().numpy())
    return {
        "acc": accuracy_score(y, preds),
        "f1": f1_score(y, preds, average="macro"),
        "mcc": matthews_corrcoef(y, preds)
    }

# =====================================================
# MAIN LOOP — STAGE 3 DL XAI
# =====================================================
for FG in FUNCTIONAL_GROUPS:
    FG = int(FG)

    print("\n" + "="*85)
    print(f"FUNCTIONAL GROUP {FG} — DEEP LEARNING STAGE-3 XAI")
    print("="*85)

    # ---- A: Extract motifs ----
    top10 = {}
    for name, model in models.items():
        motifs, weights = get_top10_IG(model, FG)
        top10[name] = motifs

    print("\nTop-10 motifs per DL model:")
    print(pd.DataFrame(top10))

    # ---- B: Stability ----
    def stab(A,B):
        A,B=set(A),set(B)
        inter = len(A&B)
        union = len(A|B)
        jac = inter/union if union>0 else 0
        pct = inter/10
        return inter, jac, pct*100

    print("\nStability (intersection, jaccard, overlap%):")
    for a,b in [("CNN","CNN-Attn"),("CNN","CNN-BiLSTM"),("CNN-Attn","CNN-BiLSTM")]:
        inter, jac, pct = stab(top10[a], top10[b])
        print(f"{a}–{b}: inter={inter}, jaccard={jac:.3f}, overlap%={pct:.1f}")

    # ---- C: Baseline ----
    baseline = {}
    for name, model in models.items():
        baseline[name] = eval_dl(model, X_test, y_test)

    print("\nBaseline performance:")
    print(pd.DataFrame(baseline).T)

    # ---- D: Fidelity (masking) ----
    fidelity = {}
    for name, model in models.items():

        masked_X = mask_sequences(X_raw_test, top10[name])
        masked_eval = eval_dl(model, masked_X, y_test)

        fidelity[name] = {
            "acc_drop": baseline[name]["acc"] - masked_eval["acc"],
            "f1_drop": baseline[name]["f1"] - masked_eval["f1"],
            "mcc_drop": baseline[name]["mcc"] - masked_eval["mcc"]
        }

    print("\nFidelity drop after masking:")
    print(pd.DataFrame(fidelity).T)

    # ---- E: Consensus ----
    s1 = set(top10["CNN"])
    s2 = set(top10["CNN-Attn"])
    s3 = set(top10["CNN-BiLSTM"])

    strict  = s1 & s2 & s3
    majority = (s1&s2)|(s1&s3)|(s2&s3)

    print("\nStrict consensus:", sorted(list(strict)) if strict else "None")
    print("Majority consensus:", sorted(list(majority)) if majority else "None")


Reconstructed vocab size: 4557

FUNCTIONAL GROUP 0 — DEEP LEARNING STAGE-3 XAI

Top-10 motifs per DL model:
      CNN CNN-Attn CNN-BiLSTM
0  cctgct   cttctg     ctgctg
1  cttctg   gtcttc     cctgct
2  ctcttc   tcttct     tcctgg
3  ctgctg   ttctgg     ctcttc
4  ctgctc   ctgcct     atgact
5  gctgct   tgtctt     ctgctc
6  tgctct   ttctgc     tgctgc
7  tcttct   ctgctg     cttctg
8  ttctgg   acttct     atggag
9  tgctgc   tctgcc     ttcctg

Stability (intersection, jaccard, overlap%):
CNN–CNN-Attn: inter=4, jaccard=0.250, overlap%=40.0
CNN–CNN-BiLSTM: inter=6, jaccard=0.429, overlap%=60.0
CNN-Attn–CNN-BiLSTM: inter=2, jaccard=0.111, overlap%=20.0

Baseline performance:
                 acc        f1       mcc
CNN         0.986928  0.984707  0.984069
CNN-Attn    0.972404  0.971100  0.966287
CNN-BiLSTM  0.988381  0.989162  0.985812

Fidelity drop after masking:
            acc_drop   f1_drop  mcc_drop
CNN         0.002179  0.001911  0.002645
CNN-Attn    0.031227  0.034205  0.038307
CNN-BiLSTM 

In [3]:
from collections import Counter

K = 6

def kmers(seq, k=K):
    return [seq[i:i+k].lower() for i in range(len(seq)-k+1)]

def build_vocab(seqs):
    all_k = []
    for s in seqs:
        all_k.extend(kmers(s, K))
    # EXACT original rule: most_common() preserves frequency order
    most_common = Counter(all_k).most_common()
    vocab = {kmer: i+1 for i, (kmer, _) in enumerate(most_common)}
    return vocab

def encode_kmer(seq, vocab):
    return [vocab[k] for k in kmers(seq, K) if k in vocab]

def pad_kmer(encoded_list, MAX_LEN=300):
    out = np.zeros((len(encoded_list), MAX_LEN), dtype=np.int64)
    for i, s in enumerate(encoded_list):
        L = min(len(s), MAX_LEN)
        out[i, :L] = s[:L]
    return out


In [35]:
X_raw = df["sequence"].values
y     = df["label"].values

vocab = build_vocab(X_raw)
X = np.stack([encode_kmer(s, vocab) for s in X_raw])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, X_raw_train, X_raw_test = train_test_split(
    X, y, X_raw, test_size=0.2, stratify=y, random_state=42
)


ValueError: all input arrays must have the same shape

In [36]:
class CNN(nn.Module):
    def __init__(self, vocab_size, num_classes=7):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, 128)
        self.convs = nn.ModuleList([
            nn.Conv1d(128, 256, 5),
            nn.Conv1d(128, 256, 7),
            nn.Conv1d(128, 256, 9)
        ])
        self.fc = nn.Linear(256*3, num_classes)

    def forward(self, x):
        x = self.embed(x).transpose(1,2)
        h = [torch.max(conv(x), dim=2)[0] for conv in self.convs]
        h = torch.cat(h, dim=1)
        return self.fc(h)
        
class CNNAttention(nn.Module):
    def __init__(self, vocab_size, num_classes=7,
                 embed_dim=256, conv_filters=256,
                 conv_kernel=15, n_heads=4, dropout=0.4):
        super().__init__()
        
        self.embed = nn.Embedding(vocab_size, embed_dim)
        
        self.conv = nn.Conv1d(embed_dim,
                              conv_filters,
                              conv_kernel,
                              padding=conv_kernel // 2)
        
        self.proj = nn.Linear(conv_filters, embed_dim)
        
        self.attn = nn.MultiheadAttention(embed_dim,
                                          n_heads,
                                          batch_first=True)
        
        self.drop = nn.Dropout(dropout)
        self.fc   = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.embed(x)                             # (B, T, E)
        x = F.relu(self.conv(x.transpose(1,2)))       # (B, C, T)
        x = self.proj(x.transpose(1,2))               # (B, T, E)
        a, _ = self.attn(x, x, x)                     # (B, T, E)
        h = a.mean(1)                                 # (B, E)
        return self.fc(self.drop(h))                  # (B, num_classes)

class CNNBiLSTM(nn.Module):
    def __init__(self, vocab_size, num_classes=7):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, 128)
        self.conv  = nn.Conv1d(128, 256, 7)
        self.lstm  = nn.LSTM(256, 256, batch_first=True, bidirectional=True)
        self.fc    = nn.Linear(512, num_classes)

    def forward(self, x):
        e = self.embed(x).transpose(1,2)
        c = self.conv(e).transpose(1,2)
        h,_ = self.lstm(c)
        h = h.mean(dim=1)
        return self.fc(h)


In [37]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
vocab_size = len(vocab)

cnn = CNN(vocab_size).to(DEVICE)
cnn.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN.pt", map_location=DEVICE))

attn = CNNAttention(vocab_size).to(DEVICE)
attn.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN-Attention.pt", map_location=DEVICE))

bilstm = CNNBiLSTM(vocab_size).to(DEVICE)
bilstm.load_state_dict(torch.load("/kaggle/input/dna-saved-models/pytorch/default/1/DL Models/DL Models/human_kmer_CNN-BiLSTM.pt", map_location=DEVICE))

cnn.eval(); attn.eval(); bilstm.eval()


RuntimeError: Error(s) in loading state_dict for CNN:
	size mismatch for embed.weight: copying a param with shape torch.Size([4470, 128]) from checkpoint, the shape in current model is torch.Size([4469, 128]).

In [26]:
TARGET_CLASS_DL = 3   # Synthetase

from captum.attr import GradientShap
import numpy as np
import torch
import torch.nn.functional as F

import torch
import numpy as np
import torch.nn.functional as F
from captum.attr import GradientShap


def top10_gradshap_global(model, X, raw_sequences,
                          class_id=TARGET_CLASS_DL,
                          k=6, topk=10,
                          max_samples=40):

    DEVICE = next(model.parameters()).device
    model.eval()

    # Disable cuDNN to allow LSTM backward
    torch.backends.cudnn.enabled = False

    Xt = torch.tensor(X, dtype=torch.long).to(DEVICE)

    # ----------------------------
    # Forward starting at embedding
    # ----------------------------
    def forward_from_emb(e):
        # CNN
        if hasattr(model, "convs"):
            x = e.transpose(1,2)
            h = [torch.max(conv(x), dim=2)[0] for conv in model.convs]
            return model.fc(torch.cat(h, dim=1))

        # CNN-Attention
        if hasattr(model, "attn"):
            c = F.relu(model.conv(e.transpose(1,2))).transpose(1,2)
            p = model.proj(c)
            a,_ = model.attn(p,p,p)
            return model.fc(model.drop(a.mean(1)))

        # CNN-BiLSTM
        if hasattr(model, "lstm"):
            c = F.relu(model.conv(e.transpose(1,2))).transpose(1,2)
            h,_ = model.lstm(c)
            return model.fc(h.mean(1))


    gs = GradientShap(forward_from_emb)

    # global 300-token importance
    global_token_scores = np.zeros(300)

    N = min(len(Xt), max_samples)

    for i in range(N):

        xi = Xt[i:i+1]  # (1, 300)
        emb = model.embed(xi)
        emb = emb.detach().clone().requires_grad_(True)

        baseline = torch.zeros_like(emb)  # zeros baseline

        # GradientShap for this one sequence
        attr = gs.attribute(
            emb,
            baselines=baseline,
            target=class_id,
            n_samples=12,
            stdevs=0.01
        )  # (1, 300, E)

        token_attr = attr.sum(dim=-1).squeeze(0).detach().cpu().numpy()
        global_token_scores += token_attr

    global_token_scores /= N   # mean over samples

    # Re-enable cuDNN
    torch.backends.cudnn.enabled = True

    # ----------------------------
    # Motif aggregation
    # ----------------------------
    motif_scores = {}

    for seq in raw_sequences[:N]:
        seq = seq.lower()
        L = len(seq)
        for i in range(L - k + 1):
            km = seq[i:i+k]
            start = max(0, min(i, 300-k))
            end   = max(0, min(i+k, 300))
            motif_scores[km] = motif_scores.get(km, 0) + global_token_scores[start:end].mean()

    sorted_motifs = sorted(motif_scores.items(), key=lambda x: x[1], reverse=True)
    return [m for m,_ in sorted_motifs[:topk]]


In [27]:
motifs_cnn  = top10_gradshap_global(cnn,   X_test, X_raw_test)
motifs_attn = top10_gradshap_global(attn,  X_test, X_raw_test)
motifs_bi   = top10_gradshap_global(bilstm, X_test, X_raw_test)

print("CNN motifs:", motifs_cnn)
print("CNN-Attention motifs:", motifs_attn)
print("CNN-BiLSTM motifs:", motifs_bi)


CNN motifs: ['ggcctg', 'agtgga', 'cagcgg', 'gtggag', 'ggagtg', 'tgggag', 'cctgaa', 'cacgca', 'ctgccg', 'gcgggg']
CNN-Attention motifs: ['ctggag', 'gaagaa', 'ggcctg', 'ctgaag', 'tgaaga', 'ggagcc', 'tgctga', 'agaaga', 'gctgga', 'ccctgg']
CNN-BiLSTM motifs: ['ctgctg', 'ggcggc', 'gaggag', 'cagcag', 'gcagcc', 'ctggac', 'tgaaga', 'gctgct', 'gctgag', 'ggagga']


In [32]:
def mask_motifs(seq, motifs):
    seq = seq.lower()
    for m in motifs:
        seq = seq.replace(m, "NNNNNN")
    return seq

def encode_masked(seq, vocab):
    return encode_kmer(seq, vocab)
    
from sklearn.metrics import accuracy_score, matthews_corrcoef
import numpy as np

def eval_dl(model, raw_sequences, true_labels, motifs=None, batch_size=16):
    """
    Memory-safe evaluation for deep learning models.
    Uses batched forward passes to avoid GPU OOM.
    Requires global 'encode_kmer' and 'vocab'.
    """

    DEVICE = next(model.parameters()).device
    model.eval()

    # 1. Mask motifs (if any)
    if motifs is not None:
        raw_masked = []
        for seq in raw_sequences:
            s = seq.lower()
            for m in motifs:
                s = s.replace(m, "N"*len(m))
            raw_masked.append(s)
    else:
        raw_masked = raw_sequences

    # 2. Encode sequences using existing vocab
    X_masked = np.stack([encode_kmer(s, vocab) for s in raw_masked])
    Xt = torch.tensor(X_masked, dtype=torch.long).to(DEVICE)

    # 3. Batched prediction
    preds = []
    with torch.no_grad():
        for i in range(0, len(Xt), batch_size):
            batch = Xt[i:i+batch_size]
            out = model(batch).argmax(dim=1).cpu().numpy()
            preds.append(out)

    preds = np.concatenate(preds)

    # 4. Metrics
    acc = accuracy_score(true_labels, preds)
    mcc = matthews_corrcoef(true_labels, preds)

    return acc, mcc


In [33]:
orig_cnn  = eval_dl(cnn,   X_raw_test, y_test)
mask_cnn  = eval_dl(cnn,   X_raw_test, y_test, motifs_cnn)

orig_attn = eval_dl(attn,  X_raw_test, y_test)
mask_attn = eval_dl(attn,  X_raw_test, y_test, motifs_attn)

orig_bi   = eval_dl(bilstm, X_raw_test, y_test)
mask_bi   = eval_dl(bilstm, X_raw_test, y_test, motifs_bi)
print("\n=== CNN FIDELITY (Top-10 Motifs) ===")
print("Original ACC:", orig_cnn[0], "Masked ACC:", mask_cnn[0], "ΔAcc =", orig_cnn[0]-mask_cnn[0])
print("Original MCC:", orig_cnn[1], "Masked MCC:", mask_cnn[1], "ΔMCC =", orig_cnn[1]-mask_cnn[1])

print("\n=== CNN-Attention FIDELITY (Top-10 Motifs) ===")
print("Original ACC:", orig_attn[0], "Masked ACC:", mask_attn[0], "ΔAcc =", orig_attn[0]-mask_attn[0])
print("Original MCC:", orig_attn[1], "Masked MCC:", mask_attn[1], "ΔMCC =", orig_attn[1]-mask_attn[1])

print("\n=== CNN-BiLSTM FIDELITY (Top-10 Motifs) ===")
print("Original ACC:", orig_bi[0], "Masked ACC:", mask_bi[0], "ΔAcc =", orig_bi[0]-mask_bi[0])
print("Original MCC:", orig_bi[1], "Masked MCC:", mask_bi[1], "ΔMCC =", orig_bi[1]-mask_bi[1])



=== CNN FIDELITY (Top-10 Motifs) ===
Original ACC: 0.20205479452054795 Masked ACC: 0.19863013698630136 ΔAcc = 0.003424657534246589
Original MCC: 0.01483197098682229 Masked MCC: 0.0011968601949607792 ΔMCC = 0.013635110791861511

=== CNN-Attention FIDELITY (Top-10 Motifs) ===
Original ACC: 0.1815068493150685 Masked ACC: 0.1780821917808219 ΔAcc = 0.003424657534246589
Original MCC: -0.0023007418133361816 Masked MCC: -0.009135370634063462 ΔMCC = 0.00683462882072728

=== CNN-BiLSTM FIDELITY (Top-10 Motifs) ===
Original ACC: 0.23401826484018265 Masked ACC: 0.1769406392694064 ΔAcc = 0.05707762557077625
Original MCC: -0.005219771470780839 Masked MCC: -0.028518811129079384 ΔMCC = 0.023299039658298545
